In [1]:
from functions import *

One way we can query phi-3 is through a locally run API server

In [13]:
from functions import *
context = "List and analyze the limitations of the studies provided within a greater context. Identify and explain any potential for future research indicated."
prompt = core_query("wind+speed+prediction", 10).loc[0, "text"]
if prompt.find("conclusion") + 4000 < len(prompt):
    prompt = prompt[prompt.find("conclusion"):prompt.find("conclusion") + 4000]
else:
    prompt = prompt[prompt.find("conclusion"):]
print("Querying API server, please be patient...")
response, context = api_server_query(context, prompt)
print(response)

invalid pdf header: b'<scri'
EOF marker not found


Bad link: https://core.ac.uk/download/pdf/6182363.pdf
Querying API server, please be patient...
The studies presented in the abstract highlight the limitations of using Markov chain models for wind speed forecasting. Specifically, the researchers note that Markov chain models require certain assumptions on the distribution of wind speed and its variations, which may not always hold true in real-world scenarios. Additionally, the researchers observed that while the Markov chain models perform well in reproducing the statistical features of wind speed, their ability to forecast wind speed at different horizon times is dependent on the time scale used. This suggests that the models may not be universally applicable and may require further optimization for specific use cases.

The use of semi-Markov models, particularly the indexed semi-Markov chain (ISMC) model, is suggested as a potential solution to these limitations. The ISMC model does not require any assumptions on the distribution o

However, it is much faster to use a serverless implementation through Azure AI  (although it's still relatively slow because of the large context length).

In [2]:
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

def research_query(prompt):
    ''' Takes a text prompt and returns phi 3.5 responses and a dictionary of token usage. '''
    load_dotenv()
    api_key = os.getenv("AZURE_INFERENCE_CREDENTIAL", '')
    if not api_key:
      raise Exception("A key should be provided to invoke the endpoint")

    client = ChatCompletionsClient(
        endpoint='https://Phi-3-5-mini-instruct-bzsru.eastus2.models.ai.azure.com',
        credential=AzureKeyCredential(api_key)
    )

    result = client.complete({
        "messages": [
          {
            "role": "user",
            "content": prompt + "List and explain within the list any potential for future research within the context of the field of research. Try to keep your response centered around 5 major gaps in the research."
          }
        ],
        "temperature": 0.8,
        "top_p": 0.95,
    })
    response = str(result.choices[0].message.content)

    # Bold the text between asterisks so it looks nice
    while "**" in response:
        first = response.find("**")
        left = response[:first]
        second = response[response.find("**") + 2:].find("**") + first + 2
        right = response[second + 2:]
        middle = response[first + 2:second]
        middle = '\033[1m' + middle + '\033[0m'
        response = left + middle + right

    response += "\n\n"
    for i in range(len(paper_df)):
        response += f"[{i + 1}] {paper_df.loc[i, 'citation']} \n\n"

    tokens_usage = {"prompt": result.usage.prompt_tokens, 
                    "completion": result.usage.completion_tokens, 
                    "total": result.usage.total_tokens}
    return response, tokens_usage

In [1]:
from functions import *
# "((stock+price)OR(valuation))AND(financial+metrics)"
# "(machine+learning)AND((price+prediction)OR(stock+price))"
print(research_query("(machine+learning)AND((price+prediction)OR(stock+price))", limit=10)[0])

# TODO: separate open access from consulted sources

 1. Integration of Multimodal Data Sources: While many studies focus on historical price data, there is potential for future research on integrating diverse data sources such as social media sentiment, economic indicators, and alternative data (e.g., satellite imagery, credit card transactions) to improve prediction accuracy. This multimodal approach could capture a broader range of factors influencing stock prices.

2. Real-time Prediction and Adaptation: Current models often rely on historical data, which may not fully capture the rapid changes in the market. Future research could explore the development of models that can adapt in real-time, using streaming data and incorporating mechanisms to update predictions as new information becomes available.

3. Explainability and Interpretability: Many advanced machine learning models, especially deep learning models, are often seen as "black boxes" with limited interpretability. Future research could focus on developing models that not onl

In [10]:
paper_df = semantic_scholar_query("(machine+learning)&((fundmantal+investing)|(stock+price))", 10)
paper_df

,title,id,abstract,tldr,field
0,A Survey on Stock Price Prediction using Machi...,21d8dcd9922e19c8aaee35eb9214ac565e06d866,"In Today's competitive environment, every Indu...","In the survey of existing works, it is found t...","Computer Science, Business"
1,Machine learning techniques for stock price pr...,77497c39a610bd3da4e91691fb53af6c7ee8d66c,None,None,"Computer Science, Computer Science, Business"
2,Stock Price Prediction using Machine Learning,1e0da55c4d06bd122a6c2a9cab51d2ab35c4ae3e,The stock market has long attracted the attent...,This paper aims to apply machine learning to a...,"Computer Science, Business"
3,A two-stage model for stock price prediction b...,fd243452d73a4d6c503d21f4be016a1f5e4e28bd,None,A novel two-stage prediction model that consis...,"Computer Science, Computer Science, Business, ..."
4,Stock Price Prediction Website Using Linear Re...,4e96fd886b35a1a9439bccf9fc4983a3b408e726,The most valuable indicator of a company’s suc...,The purpose of this paper is to demonstrate th...,"Computer Science, Business"
5,Stock Price Prediction By Applying Machine Lea...,a7c4f02a826a7e09b0ed1b64e804a799a2b901be,Stock Market Prediction is affordable access t...,Algorithm based on knowledge engineering to en...,"Computer Science, Business"
6,Machine Learning in Stock Price Analysis,ba70fb93a52f269cb094a108d7edeb0d9a4c11e1,"In recent years, people have been using machin...",This paper explains how machine learning can b...,"Computer Science, Business"
7,Forecasting the Stock Price of Listed Innovati...,a640bf71eb990cf2007f3785f335005df096cc8a,None,This study takes the Chinese market as an exam...,"Economics, Computer Science, Business"
8,Machine Learning Approaches in Stock Price Pre...,21d6d1a5377b7bc1d781e8f7ada4ed1bee76d950,Prediction of stock prices is one of the most ...,This paper draws a detailed analysis of the te...,"Physics, Computer Science, Business, Economics"
9,Prediction of Stock Price Using Machine Learni...,543f5d9726e29850b91f3f43e0e9d7d5535c6552,"In order to understand the size of economies, ...",Support Vector Machine Classifiers are applied...,"Computer Science, Business"
